#Audio Extraction from Video

In [1]:
!ffmpeg -i input_video.mp4 -q:a 0 -map a output_audio.mp3

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [2]:
pip install whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=0762b307996288adebecfe833d37d4bc39835c4998dad5006b19b22b72ca3581
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper


In [3]:
!pip install git+https://github.com/openai/whisper.git
!apt update && apt install -y ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-rft33bcf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-rft33bcf
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803669 sha256=b73ca08cca0ca0019830c513ef60b185abf1f7d89e243a7019246928bcf549f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-t4zha8u3/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://secur

#Audio Transcription with Whisper

In [4]:
import whisper

# Load the model
model = whisper.load_model("base")

# Transcribe the audio file
result = model.transcribe("output_audio.mp3", language="en")

# Access the segments
segments = result["segments"]

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 98.3MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [5]:
def write_srt(segments, output_file):
    with open(output_file, "w") as f:
        for i, segment in enumerate(segments):
            # SRT format index
            f.write(f"{i + 1}\n")

            # Start and end timestamps in HH:MM:SS,ms format
            start = segment["start"]
            end = segment["end"]
            f.write(f"{format_timestamp(start)} --> {format_timestamp(end)}\n")

            # Text of the segment
            f.write(f"{segment['text']}\n\n")

def format_timestamp(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    milliseconds = int((seconds % 1) * 1000)
    return f"{hours:02}:{minutes:02}:{int(seconds):02},{milliseconds:03}"

# Write SRT file
write_srt(segments, "subtitles_en.srt")
print("SRT file generated: subtitles_en.srt")

SRT file generated: subtitles_en.srt


#Parsing SRT Files

In [6]:
import re

def parse_srt(file_path):
    with open(file_path, "r") as f:
        srt_data = f.read()

    pattern = r"(\d+)\n(\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3})\n(.*?)\n\n"
    matches = re.findall(pattern, srt_data, re.DOTALL)
    subtitles = [{"index": m[0], "timestamp": m[1], "text": m[2]} for m in matches]
    return subtitles

english_subtitles = parse_srt("subtitles_en.srt")

In [7]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=bfc82d175d9fe681a63753de8b4d4835ec050f5b82148afcad1b345236c54b58
  Stored in directory: /root/.cache/pip/wheels/81/f2/e0/d578821d723b473d18610ea93810e4a5402463919f07e603d9
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succe

#Translating Subtitles

In [8]:
from googletrans import Translator

translator = Translator()

# Define target languages
languages = {
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Japanese": "ja",
    "Italian": "it"
}

translated_subtitles = {}
for lang, code in languages.items():
    translated_subtitles[lang] = []
    for subtitle in english_subtitles:
        translated_text = translator.translate(subtitle["text"], src="en", dest=code).text
        translated_subtitles[lang].append({
            "index": subtitle["index"],
            "timestamp": subtitle["timestamp"],
            "text": translated_text
        })

In [9]:
def generate_srt(subtitles, file_path):
    with open(file_path, "w") as f:
        for subtitle in subtitles:
            f.write(f"{subtitle['index']}\n")
            f.write(f"{subtitle['timestamp']}\n")
            f.write(f"{subtitle['text']}\n\n")

for lang, subtitles in translated_subtitles.items():
    generate_srt(subtitles, f"subtitles_{lang.lower()}.srt")

In [10]:
!pip install gtts

#Text-to-Speech Generation

In [11]:
from gtts import gTTS
import os

# Function to extract only the dialogue from subtitles (exclude timestamps)
def extract_dialogue_from_srt(srt_file):
    with open(srt_file, "r") as file:
        lines = file.readlines()

    # Extract only dialogue text (excluding timestamps)
    dialogue = []
    for line in lines:
        if "-->" not in line and line.strip() != "":
            dialogue.append(line.strip())

    # Join the dialogue into one string, which is what we need to pass to TTS
    return " ".join(dialogue)

# Function to convert text to speech and save as audio
def text_to_speech(text, language, output_filename):
    tts = gTTS(text=text, lang=language, slow=False)
    tts.save(output_filename)
    print(f"Generated audio for {language}: {output_filename}")

# Loop through each language and generate audio
languages = {
    "spanish": "es",
    "french": "fr",
    "german": "de",
    "japanese": "ja",
    "italian": "it"  # Optional language
}

# Generate audio for each translated subtitle
for lang, code in languages.items():
    srt_file = f"subtitles_{lang}.srt"  # Corresponding SRT file for the language
    if os.path.exists(srt_file):
        dialogue = extract_dialogue_from_srt(srt_file)
        output_filename = f"dubbed_{code}.mp3"
        text_to_speech(dialogue, code, output_filename)


Generated audio for es: dubbed_es.mp3
Generated audio for fr: dubbed_fr.mp3
Generated audio for de: dubbed_de.mp3
Generated audio for ja: dubbed_ja.mp3
Generated audio for it: dubbed_it.mp3


#Integrating Subtitles into Video

In [12]:
input_video = "input_video.mp4"
input_subtitle_en = "subtitles_en.srt"
input_subtitle_es = "subtitles_spanish.srt"
input_subtitle_fr = "subtitles_french.srt"
input_subtitle_de = "subtitles_german.srt"
input_subtitle_ja = "subtitles_japanese.srt"
input_subtitle_it = "subtitles_italian.srt"
output_video = "video_with_multiple_subtitles.mp4"

!ffmpeg -i {input_video} -i {input_subtitle_en} -i {input_subtitle_es} -i {input_subtitle_fr} -i {input_subtitle_de} -i {input_subtitle_ja} -i {input_subtitle_it} \
-map 0 -map 1 -map 2 -map 3 -map 4 -map 5 -map 6 \
-metadata:s:s:0 language=eng -metadata:s:s:1 language=spa -metadata:s:s:2 language=fra -metadata:s:s:3 language=deu -metadata:s:s:4 language=jpn -metadata:s:s:5 language=ita \
-c:v copy -c:a copy -c:s mov_text {output_video}


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [13]:
# Define the target languages and their respective dubbed audio and subtitle files
languages = {
    "Spanish": {
        "audio": "dubbed_es.mp3",
        "subtitle": "subtitles_spanish.srt",
        "output": "Video_Spanish.mp4"
    },
    "French": {
        "audio": "dubbed_fr.mp3",
        "subtitle": "subtitles_french.srt",
        "output": "Video_French.mp4"
    },
    "German": {
        "audio": "dubbed_de.mp3",
        "subtitle": "subtitles_german.srt",
        "output": "Video_German.mp4"
    },
    "Japanese": {
        "audio": "dubbed_ja.mp3",
        "subtitle": "subtitles_japanese.srt",
        "output": "Video_Japanese.mp4"
    },
    "Italian": {
        "audio": "dubbed_it.mp3",
        "subtitle": "subtitles_italian.srt",
        "output": "Video_Italian.mp4"
    }
}

# Process each language and generate the final video
for lang, files in languages.items():
    input_video = "input_video.mp4"
    input_audio = files["audio"]
    input_subtitle = files["subtitle"]
    output_video = files["output"]

    # Construct ffmpeg command for each language
    !ffmpeg -i {input_video} -i {input_audio} -i {input_subtitle} \
    -map 0:v -map 1:a -map 2:s \
    -metadata:s:a:0 language={lang[:3].lower()} \
    -metadata:s:s:0 language={lang[:3].lower()} \
    -c:v copy -c:a aac -c:s mov_text -strict experimental {output_video}

    print(f"Generated video for {lang}: {output_video}")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

#Download Final Videos

In [15]:
from google.colab import files

# List of files to download
files_to_download = [
    "video_with_multiple_subtitles.mp4",
    "Video_Spanish.mp4",
    "Video_French.mp4",
    "Video_German.mp4",
    "Video_Japanese.mp4",
    "Video_Italian.mp4"
]

for file in files_to_download:
    try:
        files.download(file)
        print(f"Downloaded: {file}")
    except FileNotFoundError:
        print(f"Error: {file} not found.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: video_with_multiple_subtitles.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Downloaded: Video_Japanese.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: Video_Italian.mp4


The dubbed audio tracks are not perfectly synchronized with the video, causing the audio to continue playing even after the video ends. However, the video with multiple subtitles is properly timed and works as expected.

To address the synchronization issue, we can calculate the exact duration of the original video and compare it with the duration of the dubbed audio and adjust the audio playback speed proportionally using tools like `ffmpeg` to match the video duration. Additionally, we can ensure subtitle timings are aligned with both the video and the adjusted audio to achieve seamless synchronization.